In [ ]:
"""
RMI (Recursive Model Index) Implementation in Python
Rust implementation at: https://github.com/learnedsystems/RMI
"""

import numpy as np
from typing import List, Tuple, Optional, Union, Iterator, Callable
from abc import ABC, abstractmethod
from dataclasses import dataclass
import time
import math
from enum import Enum
import bisect

# =============================================================================
# From: rmi_lib/src/models/mod.rs
# Model base classes and type definitions
# =============================================================================

class ModelDataType(Enum):
    INT = "uint64_t"
    INT128 = "uint128_t"
    FLOAT = "double"

class ModelInput:
    """From: rmi_lib/src/models/mod.rs"""
    def __init__(self, value: Union[int, float]):
        if isinstance(value, (int, np.integer)):
            self.value = int(value)
            self.is_float = False
        else:
            self.value = float(value)
            self.is_float = True
    
    def as_float(self) -> float:
        return float(self.value)
    
    def as_int(self) -> int:
        return int(self.value)

class RMITrainingData:
    """
    From: rmi_lib/src/models/mod.rs
    Training data container for RMI
    """
    def __init__(self, data: List[Tuple[Union[int, float], int]]):
        self.data = sorted(data, key=lambda x: x[0])
        self.scale = 1.0
    
    def __len__(self) -> int:
        return len(self.data)
    
    def set_scale(self, scale: float):
        self.scale = scale
    
    def get(self, idx: int) -> Tuple[Union[int, float], int]:
        key, offset = self.data[idx]
        if abs(self.scale - 1.0) > 1e-10:
            return (key, int(offset * self.scale))
        return (key, offset)
    
    def get_key(self, idx: int) -> Union[int, float]:
        return self.data[idx][0]
    
    def iter(self) -> Iterator[Tuple[Union[int, float], int]]:
        """Iterator with scale applied"""
        for key, offset in self.data:
            if abs(self.scale - 1.0) > 1e-10:
                yield (key, int(offset * self.scale))
            else:
                yield (key, offset)
    
    def iter_model_input(self) -> Iterator[Tuple[ModelInput, int]]:
        """Iterator yielding ModelInput objects"""
        for key, offset in self.iter():
            yield (ModelInput(key), offset)
    
    def iter_unique(self) -> Iterator[Tuple[Union[int, float], int]]:
        """Iterator that removes duplicate keys"""
        if len(self.data) == 0:
            return
        last_key = None
        for key, offset in self.iter():
            if last_key is None or key != last_key:
                yield (key, offset)
                last_key = key
    
    def lower_bound_by(self, cmp_func: Callable) -> int:
        """From: rmi_lib/src/models/mod.rs - binary search for lower bound"""
        size = len(self)
        if size == 0:
            return 0
        
        base = 0
        while size > 1:
            half = size // 2
            mid = base + half
            cmp_result = cmp_func(self.get(mid))
            if cmp_result < 0:  # Less
                base = mid
            size -= half
        
        cmp_result = cmp_func(self.get(base))
        base += (1 if cmp_result < 0 else 0)
        return base
    
    def soft_copy(self):
        """Create a shallow copy with same data"""
        new_data = RMITrainingData(self.data[:])
        new_data.scale = self.scale
        return new_data

class Model(ABC):
    """
    From: rmi_lib/src/models/mod.rs
    Base class for all RMI models
    """
    @abstractmethod
    def predict_to_float(self, inp: ModelInput) -> float:
        pass
    
    def predict_to_int(self, inp: ModelInput) -> int:
        return max(0, int(math.floor(self.predict_to_float(inp))))
    
    @abstractmethod
    def input_type(self) -> ModelDataType:
        pass
    
    @abstractmethod
    def output_type(self) -> ModelDataType:
        pass
    
    def needs_bounds_check(self) -> bool:
        return True
    
    def set_to_constant_model(self, constant: int) -> bool:
        return False

# =============================================================================
# From: rmi_lib/src/models/linear.rs
# Linear regression models
# =============================================================================

class LinearModel(Model):
    """From: rmi_lib/src/models/linear.rs"""
    def __init__(self, data: RMITrainingData):
        self.intercept, self.slope = self._slr(data)
    
    def _slr(self, data: RMITrainingData) -> Tuple[float, float]:
        """Simple linear regression using online algorithm"""
        mean_x = 0.0
        mean_y = 0.0
        c = 0.0
        n = 0
        m2 = 0.0
        data_size = 0
        
        for x, y in data.iter():
            n += 1
            dx = float(x) - mean_x
            mean_x += dx / n
            mean_y += (float(y) - mean_y) / n
            c += dx * (float(y) - mean_y)
            
            dx2 = float(x) - mean_x
            m2 += dx * dx2
            data_size += 1
        
        if data_size == 0:
            return (0.0, 0.0)
        if data_size == 1:
            return (mean_y, 0.0)
        
        cov = c / (n - 1)
        var = m2 / (n - 1)
        
        if var == 0.0:
            return (mean_y, 0.0)
        
        beta = cov / var
        alpha = mean_y - beta * mean_x
        
        return (alpha, beta)
    
    def predict_to_float(self, inp: ModelInput) -> float:
        return self.slope * inp.as_float() + self.intercept
    
    def input_type(self) -> ModelDataType:
        return ModelDataType.FLOAT
    
    def output_type(self) -> ModelDataType:
        return ModelDataType.FLOAT
    
    def set_to_constant_model(self, constant: int) -> bool:
        self.intercept = float(constant)
        self.slope = 0.0
        return True

class RobustLinearModel(Model):
    """From: rmi_lib/src/models/linear.rs"""
    def __init__(self, data: RMITrainingData):
        total_items = len(data)
        if total_items == 0:
            self.intercept, self.slope = (0.0, 0.0)
            return
        
        # Skip 0.01% of data from each end for robustness
        bnd = max(1, int(total_items * 0.0001))
        
        # Need at least bnd*2+1 items
        if bnd * 2 + 1 >= total_items:
            # Not enough data, use regular linear
            self.intercept, self.slope = LinearModel(data)._slr(data)
            return
        
        # Create iterator skipping first and last bnd items
        subset = []
        for i, item in enumerate(data.iter()):
            if i < bnd:
                continue
            if i >= total_items - bnd:
                break
            subset.append(item)
        
        subset_data = RMITrainingData(subset)
        self.intercept, self.slope = LinearModel(subset_data)._slr(subset_data)
    
    def predict_to_float(self, inp: ModelInput) -> float:
        return self.slope * inp.as_float() + self.intercept
    
    def input_type(self) -> ModelDataType:
        return ModelDataType.FLOAT
    
    def output_type(self) -> ModelDataType:
        return ModelDataType.FLOAT
    
    def set_to_constant_model(self, constant: int) -> bool:
        self.intercept = float(constant)
        self.slope = 0.0
        return True

# =============================================================================
# From: rmi_lib/src/models/linear_spline.rs
# Linear spline model
# =============================================================================

class LinearSplineModel(Model):
    """From: rmi_lib/src/models/linear_spline.rs"""
    def __init__(self, data: RMITrainingData):
        self.intercept, self.slope = self._linear_splines(data)
    
    def _linear_splines(self, data: RMITrainingData) -> Tuple[float, float]:
        if len(data) == 0:
            return (0.0, 0.0)
        if len(data) == 1:
            return (float(data.get(0)[1]), 0.0)
        
        first_pt = data.get(0)
        last_pt = data.get(len(data) - 1)
        
        if first_pt[0] == last_pt[0]:
            return (float(data.get(0)[1]), 0.0)
        
        slope = (float(first_pt[1]) - float(last_pt[1])) / (float(first_pt[0]) - float(last_pt[0]))
        intercept = float(first_pt[1]) - slope * float(first_pt[0])
        
        return (intercept, slope)
    
    def predict_to_float(self, inp: ModelInput) -> float:
        return self.slope * inp.as_float() + self.intercept
    
    def input_type(self) -> ModelDataType:
        return ModelDataType.FLOAT
    
    def output_type(self) -> ModelDataType:
        return ModelDataType.FLOAT
    
    def set_to_constant_model(self, constant: int) -> bool:
        self.intercept = float(constant)
        self.slope = 0.0
        return True

# =============================================================================
# From: rmi_lib/src/models/cubic_spline.rs
# Cubic spline model
# =============================================================================

class CubicSplineModel(Model):
    """From: rmi_lib/src/models/cubic_spline.rs"""
    def __init__(self, data: RMITrainingData):
        self.a, self.b, self.c, self.d = self._cubic(data)
        
        # Check against linear model - sometimes cubic doesn't work well
        linear = LinearSplineModel(data)
        
        our_error = 0.0
        lin_error = 0.0
        
        for x, y in data.iter_model_input():
            c_pred = self.predict_to_float(x)
            l_pred = linear.predict_to_float(x)
            
            our_error += abs(c_pred - float(y))
            lin_error += abs(l_pred - float(y))
        
        if lin_error < our_error:
            # Use linear instead
            self.a = 0.0
            self.b = 0.0
            self.c = linear.slope
            self.d = linear.intercept
    
    def _cubic(self, data: RMITrainingData) -> Tuple[float, float, float, float]:
        if len(data) == 0:
            return (0.0, 0.0, 1.0, 0.0)
        if len(data) == 1:
            return (0.0, 0.0, 0.0, float(data.get(0)[1]))
        
        # Check for unique values
        candidate = data.get(0)[0]
        uniq = any(x != candidate for x, _ in data.iter())
        if not uniq:
            return (0.0, 0.0, 0.0, float(data.get(0)[1]))
        
        first_pt = data.get(0)
        last_pt = data.get(len(data) - 1)
        xmin, ymin = float(first_pt[0]), float(first_pt[1])
        xmax, ymax = float(last_pt[0]), float(last_pt[1])
        
        x1, y1 = 0.0, 0.0
        x2, y2 = 1.0, 1.0
        
        # Find first point with scaled x > 0
        m1 = None
        for xn, yn in data.iter():
            sxn = (float(xn) - xmin) / (xmax - xmin)
            if sxn > 0.0:
                syn = (float(yn) - ymin) / (ymax - ymin)
                m1 = (syn - y1) / (sxn - x1)
                break
        
        if m1 is None:
            m1 = 0.0
        
        # Find last point with scaled x < 1
        m2 = None
        for i in range(len(data) - 1, -1, -1):
            xp, yp = data.get(i)
            sxp = (float(xp) - xmin) / (xmax - xmin)
            if sxp < 1.0:
                syp = (float(yp) - ymin) / (ymax - ymin)
                m2 = (y2 - syp) / (x2 - sxp)
                break
        
        if m2 is None:
            m2 = 0.0
        
        # Keep monotonic
        if m1**2 + m2**2 > 9.0:
            tau = 3.0 / math.sqrt(m1**2 + m2**2)
            m1 *= tau
            m2 *= tau
        
        # Compute coefficients
        a = (m1 + m2 - 2.0) / (xmax - xmin)**3
        b = -(xmax * (2.0*m1 + m2 - 3.0) + xmin * (m1 + 2.0*m2 - 3.0)) / (xmax - xmin)**3
        c = (m1*xmax**2 + m2*xmin**2 + xmax*xmin*(2.0*m1 + 2.0*m2 - 6.0)) / (xmax - xmin)**3
        d = -xmin * (m1*xmax**2 + xmax*xmin*(m2 - 3.0) + xmin**2) / (xmax - xmin)**3
        
        a *= ymax - ymin
        b *= ymax - ymin
        c *= ymax - ymin
        d *= ymax - ymin
        d += ymin
        
        return (a, b, c, d)
    
    def predict_to_float(self, inp: ModelInput) -> float:
        val = inp.as_float()
        # Use FMA-like computation
        v1 = self.a * val + self.b
        v2 = v1 * val + self.c
        v3 = v2 * val + self.d
        return v3
    
    def input_type(self) -> ModelDataType:
        return ModelDataType.FLOAT
    
    def output_type(self) -> ModelDataType:
        return ModelDataType.FLOAT
    
    def set_to_constant_model(self, constant: int) -> bool:
        self.a = 0.0
        self.b = 0.0
        self.c = 0.0
        self.d = float(constant)
        return True

# =============================================================================
# From: rmi_lib/src/models/radix.rs
# Radix models
# =============================================================================

def num_bits(largest_target: int) -> int:
    """From: rmi_lib/src/models/utils.rs"""
    nbits = 0
    while (1 << (nbits + 1)) - 1 <= largest_target:
        nbits += 1
    assert nbits >= 1
    return nbits

def common_prefix_size(data: RMITrainingData) -> int:
    """From: rmi_lib/src/models/utils.rs"""
    any_ones = 0
    no_ones = (1 << 64) - 1
    
    for x, _ in data.iter_model_input():
        val = x.as_int()
        any_ones |= val
        no_ones &= val
    
    any_zeros = ~no_ones & ((1 << 64) - 1)
    prefix_bits = any_zeros ^ any_ones
    
    # Count leading zeros
    prefix_bits &= ((1 << 64) - 1)
    if prefix_bits == 0:
        return 64
    
    leading_zeros = 0
    test_bit = 1 << 63
    while test_bit > 0 and (~prefix_bits & test_bit):
        leading_zeros += 1
        test_bit >>= 1
    
    return leading_zeros

class RadixModel(Model):
    """From: rmi_lib/src/models/radix.rs"""
    def __init__(self, data: RMITrainingData):
        if len(data) == 0:
            self.left_shift = 0
            self.num_bits = 0
            return
        
        largest_value = max(y for _, y in data.iter())
        bits = num_bits(largest_value)
        common_prefix = common_prefix_size(data)
        
        self.left_shift = common_prefix
        self.num_bits = bits
    
    def predict_to_int(self, inp: ModelInput) -> int:
        as_int = inp.as_int()
        res = (as_int << self.left_shift) >> (64 - self.num_bits)
        return res & ((1 << 64) - 1)
    
    def predict_to_float(self, inp: ModelInput) -> float:
        return float(self.predict_to_int(inp))
    
    def input_type(self) -> ModelDataType:
        return ModelDataType.INT
    
    def output_type(self) -> ModelDataType:
        return ModelDataType.INT
    
    def needs_bounds_check(self) -> bool:
        return False

class RadixTable(Model):
    """From: rmi_lib/src/models/radix.rs"""
    def __init__(self, data: RMITrainingData, bits: int):
        self.prefix_bits = common_prefix_size(data)
        self.table_bits = bits
        self.hint_table = [0] * (1 << bits)
        
        last_radix = 0
        for inp, y in data.iter_model_input():
            x = inp.as_int()
            num_bits = 0 if self.prefix_bits + bits > 64 else 64 - (self.prefix_bits + bits)
            current_radix = ((x << self.prefix_bits) >> self.prefix_bits) >> num_bits
            
            if current_radix == last_radix:
                continue
            
            self.hint_table[int(current_radix)] = y
            
            for i in range(int(last_radix) + 1, int(current_radix)):
                self.hint_table[i] = y
            
            last_radix = current_radix
        
        for i in range(int(last_radix) + 1, len(self.hint_table)):
            self.hint_table[i] = len(self.hint_table)
    
    def predict_to_int(self, inp: ModelInput) -> int:
        as_int = inp.as_int()
        num_bits = 0 if self.prefix_bits + self.table_bits > 64 else 64 - (self.prefix_bits + self.table_bits)
        res = ((as_int << self.prefix_bits) >> self.prefix_bits) >> num_bits
        idx = self.hint_table[int(res)]
        return idx
    
    def predict_to_float(self, inp: ModelInput) -> float:
        return float(self.predict_to_int(inp))
    
    def input_type(self) -> ModelDataType:
        return ModelDataType.INT
    
    def output_type(self) -> ModelDataType:
        return ModelDataType.INT
    
    def needs_bounds_check(self) -> bool:
        return False

# =============================================================================
# From: rmi_lib/src/train/lower_bound_correction.rs
# Lower bound correction for empty models
# =============================================================================

class LowerBoundCorrection:
    """From: rmi_lib/src/train/lower_bound_correction.rs"""
    def __init__(self, pred_func: Callable, num_leaf_models: int, data: RMITrainingData):
        self.first = [None] * num_leaf_models
        self.last = [None] * num_leaf_models
        self.next = [(0, 0)] * num_leaf_models
        self.prev = [(0, 0)] * num_leaf_models
        self.run_lengths = [0] * num_leaf_models
        
        last_target = 0
        current_run_length = 0
        current_run_key = data.get_key(0)
        
        for x, y in data.iter():
            leaf_idx = pred_func(x)
            target = min(num_leaf_models - 1, leaf_idx)
            
            if target == last_target and x == current_run_key:
                current_run_length += 1
            elif target != last_target or x != current_run_key:
                self.run_lengths[last_target] = max(
                    self.run_lengths[last_target], current_run_length
                )
                current_run_length = 1
                current_run_key = x
                last_target = target
            
            if self.first[target] is None:
                self.first[target] = (y, x)
            self.last[target] = (y, x)
        
        # Compute next_for_leaf
        idx = 0
        while idx < num_leaf_models:
            next_found = None
            for i in range(idx + 1, num_leaf_models):
                if self.first[i] is not None:
                    next_found = (i, self.first[i])
                    break
            
            if next_found:
                next_leaf_idx, val = next_found
                for i in range(idx, next_leaf_idx):
                    self.next[i] = val
                idx = next_leaf_idx
            else:
                for i in range(idx, num_leaf_models):
                    self.next[i] = (len(data), data.get_key(len(data) - 1) if len(data) > 0 else 0)
                break
        
        # Compute prev_for_leaf
        idx = num_leaf_models - 1
        while idx > 0:
            prev_found = None
            for i in range(idx - 1, -1, -1):
                if self.last[i] is not None:
                    prev_found = (i, self.last[i])
                    break
            
            if prev_found:
                prev_leaf_idx, val = prev_found
                for i in range(prev_leaf_idx + 1, idx + 1):
                    self.prev[i] = val
                idx = prev_leaf_idx
            else:
                break
    
    def first_key(self, leaf_idx: int) -> Optional[Union[int, float]]:
        return self.first[leaf_idx][1] if self.first[leaf_idx] else None
    
    def last_key(self, leaf_idx: int) -> Optional[Union[int, float]]:
        return self.last[leaf_idx][1] if self.last[leaf_idx] else None
    
    def next_index(self, leaf_idx: int) -> int:
        return self.next[leaf_idx][0]
    
    def prev_key(self, leaf_idx: int) -> Union[int, float]:
        return self.prev[leaf_idx][1]
    
    def longest_run(self, leaf_idx: int) -> int:
        return self.run_lengths[leaf_idx]

# =============================================================================
# From: rmi_lib/src/train/two_layer.rs
# Two-layer RMI training
# =============================================================================

def error_between(v1: int, v2: int, max_pred: int) -> int:
    """From: rmi_lib/src/train/two_layer.rs"""
    pred1 = min(v1, max_pred)
    pred2 = min(v2, max_pred)
    return max(pred1, pred2) - min(pred1, pred2)

def train_model(model_type: str, data: RMITrainingData) -> Model:
    """From: rmi_lib/src/train/mod.rs"""
    model_map = {
        'linear': LinearModel,
        'robust_linear': RobustLinearModel,
        'linear_spline': LinearSplineModel,
        'cubic': CubicSplineModel,
        'radix': RadixModel,
    }
    
    if model_type.startswith('radix') and model_type != 'radix':
        bits = int(model_type[5:])
        return RadixTable(data, bits)
    
    if model_type not in model_map:
        raise ValueError(f"Unknown model type: {model_type}")
    
    return model_map[model_type](data)

@dataclass
class TrainedRMI:
    """From: rmi_lib/src/train/mod.rs"""
    num_rmi_rows: int
    num_data_rows: int
    model_avg_error: float
    model_avg_l2_error: float
    model_avg_log2_error: float
    model_max_error: int
    model_max_error_idx: int
    model_max_log2_error: float
    last_layer_max_l1s: List[int]
    rmi: List[List[Model]]
    models: str
    branching_factor: int
    build_time: float
    
    def predict(self, key: Union[int, float]) -> Tuple[int, int]:
        """Predict position and error for a key"""
        inp = ModelInput(key)
        
        # First layer
        model_idx = self.rmi[0][0].predict_to_int(inp)
        
        # Second layer
        model_idx = min(len(self.rmi[1]) - 1, model_idx)
        prediction = self.rmi[1][model_idx].predict_to_int(inp)
        error = self.last_layer_max_l1s[model_idx]
        
        return (prediction, error)

def build_models_from(
    data: RMITrainingData,
    top_model: Model,
    model_type: str,
    start_idx: int,
    end_idx: int,
    first_model_idx: int,
    num_models: int
) -> List[Model]:
    """From: rmi_lib/src/train/two_layer.rs"""
    
    assert end_idx > start_idx
    assert end_idx <= len(data)
    assert start_idx <= len(data)
    
    leaf_models = []
    second_layer_data = []
    last_target = first_model_idx
    
    # Get bounded iterator
    for i, (x, y) in enumerate(data.iter()):
        if i < start_idx:
            continue
        if i >= end_idx:
            break
        
        model_pred = top_model.predict_to_int(ModelInput(x))
        target = min(first_model_idx + num_models - 1, model_pred)
        assert target >= last_target
        
        if target > last_target:
            # Train previous model
            last_item = second_layer_data[-1] if second_layer_data else None
            second_layer_data.append((x, y))
            
            container = RMITrainingData(second_layer_data)
            leaf_model = train_model(model_type, container)
            leaf_models.append(leaf_model)
            
            # Add empty models for skipped indices
            for _ in range(last_target + 1, target):
                empty_data = RMITrainingData([])
                leaf_models.append(train_model(model_type, empty_data))
            
            assert len(leaf_models) + first_model_idx == target
            
            second_layer_data = []
            if last_item:
                second_layer_data.append(last_item)
        
        second_layer_data.append((x, y))
        last_target = target
    
    # Train last model
    assert second_layer_data
    container = RMITrainingData(second_layer_data)
    leaf_model = train_model(model_type, container)
    leaf_models.append(leaf_model)
    
    # Add remaining empty models
    for _ in range(last_target + 1, first_model_idx + num_models):
        empty_data = RMITrainingData([])
        leaf_models.append(train_model(model_type, empty_data))
    
    assert len(leaf_models) == num_models
    return leaf_models

def train_two_layer(
    data: RMITrainingData,
    layer1_model: str,
    layer2_model: str,
    num_leaf_models: int
) -> TrainedRMI:
    """From: rmi_lib/src/train/two_layer.rs"""
    
    num_rows = len(data)
    
    print(f"Training top-level {layer1_model} model...")
    data.set_scale(num_leaf_models / num_rows)
    top_model = train_model(layer1_model, data)
    
    print(f"Training second-level {layer2_model} models (num models = {num_leaf_models})...")
    data.set_scale(1.0)
    
    # Find split point near middle
    midpoint_model = num_leaf_models // 2
    split_idx = data.lower_bound_by(
        lambda x: -1 if top_model.predict_to_int(ModelInput(x[0])) < midpoint_model 
                  else (1 if top_model.predict_to_int(ModelInput(x[0])) > midpoint_model else 0)
    )
    
    # Build leaf models
    if split_idx >= len(data):
        leaf_models = build_models_from(
            data, top_model, layer2_model, 0, len(data), 0, num_leaf_models
        )
    else:
        split_idx_target = min(
            num_leaf_models - 1,
            top_model.predict_to_int(ModelInput(data.get_key(split_idx)))
        )
        
        first_half_models = split_idx_target
        second_half_models = num_leaf_models - split_idx_target
        
        # Build first half
        hf1 = build_models_from(
            data, top_model, layer2_model, 0, split_idx, 0, first_half_models
        )
        
        # Build second half
        hf2 = build_models_from(
            data, top_model, layer2_model, split_idx + 1, len(data),
            split_idx_target, second_half_models
        )
        
        leaf_models = hf1 + hf2
    
    print("Computing lower bound corrections...")
    lb_corrections = LowerBoundCorrection(
        lambda x: top_model.predict_to_int(ModelInput(x)),
        num_leaf_models,
        data
    )
    
    print("Fixing empty models...")
    # Replace empty models with constants
    for idx in range(num_leaf_models - 1):
        if lb_corrections.first_key(idx) is None:
            upper_bound = lb_corrections.next_index(idx)
            leaf_models[idx].set_to_constant_model(upper_bound)
    
    print("Computing last level errors...")
    # Compute errors for each leaf model
    last_layer_max_l1s = [(0, 0)] * num_leaf_models
    
    for x, y in data.iter_model_input():
        leaf_idx = top_model.predict_to_int(x)
        target = min(num_leaf_models - 1, leaf_idx)
        
        pred = leaf_models[target].predict_to_int(x)
        err = error_between(pred, y, len(data))
        
        cur_count, cur_max = last_layer_max_l1s[target]
        last_layer_max_l1s[target] = (cur_count + 1, max(err, cur_max))
    
    # Adjust errors for lower bound correctness
    for leaf_idx in range(num_leaf_models):
        curr_err = last_layer_max_l1s[leaf_idx][1]
        
        # Upper error
        idx_of_next, key_of_next = lb_corrections.next[leaf_idx]
        key_minus_eps = key_of_next - (1 if isinstance(key_of_next, int) else 1e-10)
        pred = leaf_models[leaf_idx].predict_to_int(ModelInput(key_minus_eps))
        upper_error = error_between(pred, idx_of_next + 1, len(data))
        
        # Lower error
        first_key_before = lb_corrections.prev_key(leaf_idx)
        prev_idx = 0 if leaf_idx == 0 else leaf_idx - 1
        first_idx = lb_corrections.next_index(prev_idx)
        
        key_plus_eps = first_key_before + (1 if isinstance(first_key_before, int) else 1e-10)
        pred = leaf_models[leaf_idx].predict_to_int(ModelInput(key_plus_eps))
        lower_error = error_between(pred, first_idx, len(data))
        
        new_err = max(curr_err, upper_error, lower_error) + lb_corrections.longest_run(leaf_idx)
        
        num_items = last_layer_max_l1s[leaf_idx][0]
        last_layer_max_l1s[leaf_idx] = (num_items, new_err)
    
    print("Evaluating RMI...")
    # Compute statistics
    model_max_error_idx, (_, model_max_error) = max(
        enumerate(last_layer_max_l1s), key=lambda x: x[1][1]
    )
    
    total_items = sum(n for n, _ in last_layer_max_l1s)
    model_avg_error = sum(n * err for n, err in last_layer_max_l1s) / total_items
    model_avg_l2_error = sum((n * err) ** 2 for n, err in last_layer_max_l1s) / total_items
    model_avg_log2_error = sum(n * math.log2(2 * err + 2) for n, err in last_layer_max_l1s) / total_items
    model_max_log2_error = math.log2(model_max_error) if model_max_error > 0 else 0.0
    
    final_errors = [err for _, err in last_layer_max_l1s]
    
    return TrainedRMI(
        num_rmi_rows=len(data),
        num_data_rows=len(data),
        model_avg_error=model_avg_error,
        model_avg_l2_error=model_avg_l2_error,
        model_avg_log2_error=model_avg_log2_error,
        model_max_error=model_max_error,
        model_max_error_idx=model_max_error_idx,
        model_max_log2_error=model_max_log2_error,
        last_layer_max_l1s=final_errors,
        rmi=[[top_model], leaf_models],
        models=f"{layer1_model},{layer2_model}",
        branching_factor=num_leaf_models,
        build_time=0.0
    )

# =============================================================================
# From: rmi_lib/src/train/mod.rs
# Main training function
# =============================================================================

def train(data: RMITrainingData, model_spec: str, branch_factor: int) -> TrainedRMI:
    """From: rmi_lib/src/train/mod.rs"""
    
    start_time = time.time()
    
    model_list = model_spec.split(',')
    if len(model_list) != 2:
        raise ValueError("Only two-layer RMIs are currently supported")
    
    layer1_model, layer2_model = model_list
    
    result = train_two_layer(data, layer1_model, layer2_model, branch_factor)
    
    build_time = time.time() - start_time
    result.build_time = build_time
    
    print(f"\nRMI Training Complete!")
    print(f"Build time: {build_time:.3f}s")
    print(f"Average error: {result.model_avg_error:.2f}")
    print(f"Max error: {result.model_max_error}")
    print(f"Average log2 error: {result.model_avg_log2_error:.3f}")
    print(f"Max log2 error: {result.model_max_log2_error:.3f}")
    
    return result

# =============================================================================
# Example usage and testing
# =============================================================================

def load_binary_data(filename: str) -> RMITrainingData:
    """
    Load binary data file in the format expected by RMI:
    - First 8 bytes: number of items (uint64, little endian)
    - Remaining bytes: data items (uint64 or uint32, little endian)
    """
    import struct
    
    with open(filename, 'rb') as f:
        # Read number of items
        num_items_bytes = f.read(8)
        num_items = struct.unpack('<Q', num_items_bytes)[0]
        
        # Determine data type from filename
        if 'uint32' in filename:
            fmt = '<I'
            item_size = 4
        elif 'uint64' in filename:
            fmt = '<Q'
            item_size = 8
        else:
            raise ValueError("Filename must contain 'uint32' or 'uint64'")
        
        # Read all data
        data = []
        for i in range(num_items):
            item_bytes = f.read(item_size)
            if len(item_bytes) < item_size:
                break
            value = struct.unpack(fmt, item_bytes)[0]
            data.append((value, i))
    
    return RMITrainingData(data)

def create_synthetic_data(n: int, distribution: str = 'uniform') -> RMITrainingData:
    """Create synthetic sorted data for testing"""
    
    if distribution == 'uniform':
        keys = np.sort(np.random.randint(0, n * 10, size=n))
    elif distribution == 'normal':
        keys = np.sort(np.random.normal(n * 5, n, size=n).astype(int))
        keys = np.maximum(keys, 0)  # Ensure non-negative
    elif distribution == 'lognormal':
        keys = np.sort(np.random.lognormal(10, 2, size=n).astype(int))
    elif distribution == 'exponential':
        keys = np.sort(np.random.exponential(n / 10, size=n).astype(int))
    else:
        raise ValueError(f"Unknown distribution: {distribution}")
    
    # Create (key, position) pairs
    data = [(int(k), i) for i, k in enumerate(keys)]
    return RMITrainingData(data)

def benchmark_rmi(rmi: TrainedRMI, test_keys: List[int], actual_data: List[Tuple[int, int]]):
    """
    Benchmark RMI performance
    From: general benchmarking approach
    """
    print("\n=== Benchmarking RMI ===")
    
    # Create sorted array for binary search comparison
    sorted_keys = [k for k, _ in actual_data]
    
    # Test predictions
    total_search_range = 0
    correct_predictions = 0
    
    for key in test_keys:
        pred, err = rmi.predict(key)
        
        # Check if prediction is within error bound
        lower = max(0, pred - err)
        upper = min(len(actual_data) - 1, pred + err)
        total_search_range += (upper - lower + 1)
        
        # Find actual position
        actual_pos = bisect.bisect_left(sorted_keys, key)
        
        if lower <= actual_pos <= upper:
            correct_predictions += 1
    
    avg_search_range = total_search_range / len(test_keys)
    accuracy = 100 * correct_predictions / len(test_keys)
    
    print(f"Tested {len(test_keys)} keys")
    print(f"Average search range: {avg_search_range:.2f}")
    print(f"Prediction accuracy: {accuracy:.2f}%")
    print(f"Avg log2(search range): {math.log2(avg_search_range + 1):.3f}")
    
    return {
        'avg_search_range': avg_search_range,
        'accuracy': accuracy,
        'total_tested': len(test_keys)
    }

# =============================================================================
# Main demonstration
# =============================================================================

def main():
    """Demonstrate RMI training and usage"""
    
    print("=" * 70)
    print("RMI (Recursive Model Index) Implementation")
    print("=" * 70)
    
    # Create synthetic dataset
    print("\n1. Creating synthetic dataset...")
    n = 1_000_000
    data = create_synthetic_data(n, distribution='normal')
    print(f"Created {len(data)} data points")
    
    # Train RMI with different configurations
    configs = [
        ("linear,linear", 1000),
        ("robust_linear,linear", 1000),
        ("cubic,linear", 1000),
        ("linear_spline,linear", 1000),
    ]
    
    results = []
    
    for model_spec, branch_factor in configs:
        print(f"\n{'=' * 70}")
        print(f"2. Training RMI: {model_spec} with branching factor {branch_factor}")
        print(f"{'=' * 70}")
        
        rmi = train(data.soft_copy(), model_spec, branch_factor)
        
        # Test on random keys
        print("\n3. Testing predictions...")
        test_keys = [data.get_key(i) for i in np.random.randint(0, len(data), size=1000)]
        
        benchmark_results = benchmark_rmi(rmi, test_keys, data.data)
        
        results.append({
            'config': model_spec,
            'branching_factor': branch_factor,
            'build_time': rmi.build_time,
            'avg_error': rmi.model_avg_error,
            'max_error': rmi.model_max_error,
            'avg_log2_error': rmi.model_avg_log2_error,
            'avg_search_range': benchmark_results['avg_search_range']
        })
    
    # Print comparison table
    print("\n" + "=" * 70)
    print("RESULTS COMPARISON")
    print("=" * 70)
    print(f"{'Config':<25} {'Branch':<8} {'Build(s)':<10} {'AvgErr':<10} {'MaxErr':<10} {'AvgLog2':<10} {'SearchRng':<10}")
    print("-" * 70)
    
    for r in results:
        print(f"{r['config']:<25} {r['branching_factor']:<8} {r['build_time']:<10.3f} "
              f"{r['avg_error']:<10.2f} {r['max_error']:<10} {r['avg_log2_error']:<10.3f} "
              f"{r['avg_search_range']:<10.2f}")
    
    print("\n" + "=" * 70)
    print("Done!")
    print("=" * 70)

if __name__ == "__main__":
    main()

RMI (Recursive Model Index) Implementation
Translated from Rust implementation

1. Creating synthetic dataset...
Created 1000000 data points

2. Training RMI: linear,linear with branching factor 1000
Training top-level linear model...
Training second-level linear models (num models = 1000)...
Computing lower bound corrections...
Fixing empty models...
Computing last level errors...
Evaluating RMI...

RMI Training Complete!
Build time: 5.543s
Average error: 1841.99
Max error: 38888
Average log2 error: 6.352
Max log2 error: 15.247

3. Testing predictions...

=== Benchmarking RMI ===
Tested 1000 keys
Average search range: 3015.47
Prediction accuracy: 99.80%
Avg log2(search range): 11.559

2. Training RMI: robust_linear,linear with branching factor 1000
Training top-level robust_linear model...
Training second-level linear models (num models = 1000)...
Computing lower bound corrections...
Fixing empty models...
Computing last level errors...
Evaluating RMI...

RMI Training Complete!
Build 